In [3]:
import torch
import numpy as np
import random
import json
import logging
import os
import pickle
import pandas as pd
import importlib
from tqdm import tqdm
#import model2
from sklearn.model_selection import train_test_split
import statistics

In [4]:
SEED = 12345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
local_rank = -1
fp16 = False
if local_rank == -1:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(local_rank)
    device = torch.device("cuda", local_rank)
    n_gpu = 1
  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend='nccl')
logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(device, n_gpu, bool(local_rank != -1), fp16))
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

07/19/2023 05:37:40 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False


In [6]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'

In [8]:
#ehr_data = pd.read_csv('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/mg_sma_1/ehr_data/records.csv', dtype={'claim_id':str})
proc_dict = pickle.load(open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2/ehr_data/proc_dict','rb'))
diag_dict = pickle.load(open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2/ehr_data/diag_dict','rb'))
prod_dict = pickle.load(open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2/ehr_data/prod_dict','rb'))
embedding_dict = pickle.load(open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/embedding_dict','rb'))

In [9]:
#ehr_data1 = pd.read_csv('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/mg_sma_1/ehr_data/records.csv', dtype={'claim_id':str})
#ehr_data1

In [10]:
#ehr_data1.dtypes

In [11]:
mg_data = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/mg_records.csv', dtype={'claim_id':str,'origl_prod_svc_qlfr_cd':str,'diag_cd':str,'product_id':str})
#pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_1/ehr_data/diag_list.csv',dtype={'diag_cd':str})

07/19/2023 05:56:34 - INFO - botocore.credentials -   Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [12]:
mg_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,743206876,M54.2,97110,69.0,M,NaN,DUMMY_NCT_01
1,1247265182,Z46.6,99212,82.0,F,NaN,DUMMY_NCT_01
2,794359068,R74.01,99214,79.0,M,NaN,DUMMY_NCT_01
3,1011422753,I10,99214,64.0,F,NaN,DUMMY_NCT_01
4,9631592600,D59.3,96413,68.0,M,NaN,DUMMY_NCT_01
...,...,...,...,...,...,...,...
10734215,374739786,I67.9,97116,86.0,F,NaN,DUMMY_NCT_01
10734216,865433928,I25.10,93010,80.0,M,NaN,DUMMY_NCT_01
10734217,2797947170,M54.2,97110,56.0,F,NaN,DUMMY_NCT_01
10734218,692729515,M19.90,99233,78.0,F,NaN,DUMMY_NCT_01


In [13]:
mg_data.dtypes

claim_id                   object
diag_cd                    object
origl_prod_svc_qlfr_cd     object
pat_age_yr_nbr            float64
pat_gender_cd              object
product_id                 object
nct_id                     object
dtype: object

In [14]:
sma_data = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/sma_records.csv', dtype={'claim_id':str,'origl_prod_svc_qlfr_cd':str,'diag_cd':str,'product_id':str})


In [15]:
sma_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1260528814,J47.1,99233,11.0,M,NaN,NCT05232929
1,2268926808,I21.4,83735,72.0,M,NaN,NCT05232929
2,197296076,G12.9,T1003,20.0,M,NaN,NCT05232929
3,389979281,R10.11,83690,27.0,F,NaN,NCT05232929
4,599397345,I50.32,81003,68.0,F,NaN,NCT05232929
...,...,...,...,...,...,...,...
2746248,6974367544,Z93.0,A4450,4.0,M,NaN,NCT05232929
2746249,1665761185,E66.3,80048,61.0,F,NaN,NCT05232929
2746250,1652961827,E78.5,84439,40.0,F,NaN,NCT05232929
2746251,786601104,G70.9,C1889,61.0,F,NaN,NCT05232929


In [16]:
ms1_data = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms1_records.csv', dtype={'claim_id':str,'origl_prod_svc_qlfr_cd':str,'diag_cd':str,'product_id':str})


In [17]:
ms1_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1731468274,R20.9,80053,33.0,F,NaN,NCT04699747
1,712193600,Z20.822,99284,65.0,F,NaN,NCT04699747
2,955776860,U07.1,E1392,56.0,F,NaN,NCT04699747
3,9704756583,E44.0,NaN,47.0,F,NaN,NCT04699747
4,3353246201,G35,96415,36.0,F,NaN,NCT04699747
...,...,...,...,...,...,...,...
60246361,120789064,M47.816,1125F,70.0,F,NaN,NCT04699747
60246362,141117537,J96.90,E0431,70.0,F,NaN,NCT04699747
60246363,132681533,M54.12,97140,59.0,M,NaN,NCT04699747
60246364,30220098,N39.0,87186,NaN,M,NaN,NCT04699747


In [18]:
ms2_data = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms1_records.csv', dtype={'claim_id':str,'origl_prod_svc_qlfr_cd':str,'diag_cd':str,'product_id':str})


In [19]:
ms2_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1731468274,R20.9,80053,33.0,F,NaN,NCT04699747
1,712193600,Z20.822,99284,65.0,F,NaN,NCT04699747
2,955776860,U07.1,E1392,56.0,F,NaN,NCT04699747
3,9704756583,E44.0,NaN,47.0,F,NaN,NCT04699747
4,3353246201,G35,96415,36.0,F,NaN,NCT04699747
...,...,...,...,...,...,...,...
60246361,120789064,M47.816,1125F,70.0,F,NaN,NCT04699747
60246362,141117537,J96.90,E0431,70.0,F,NaN,NCT04699747
60246363,132681533,M54.12,97140,59.0,M,NaN,NCT04699747
60246364,30220098,N39.0,87186,NaN,M,NaN,NCT04699747


In [20]:
ms2_nct=['NCT04650321']*60246366

In [21]:
ms2_data.nct_id=ms2_nct

In [22]:
ms2_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1731468274,R20.9,80053,33.0,F,NaN,NCT04650321
1,712193600,Z20.822,99284,65.0,F,NaN,NCT04650321
2,955776860,U07.1,E1392,56.0,F,NaN,NCT04650321
3,9704756583,E44.0,NaN,47.0,F,NaN,NCT04650321
4,3353246201,G35,96415,36.0,F,NaN,NCT04650321
...,...,...,...,...,...,...,...
60246361,120789064,M47.816,1125F,70.0,F,NaN,NCT04650321
60246362,141117537,J96.90,E0431,70.0,F,NaN,NCT04650321
60246363,132681533,M54.12,97140,59.0,M,NaN,NCT04650321
60246364,30220098,N39.0,87186,NaN,M,NaN,NCT04650321


In [23]:
ms3_data = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms1_records.csv', dtype={'claim_id':str,'origl_prod_svc_qlfr_cd':str,'diag_cd':str,'product_id':str})


In [24]:
ms3_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1731468274,R20.9,80053,33.0,F,NaN,NCT04699747
1,712193600,Z20.822,99284,65.0,F,NaN,NCT04699747
2,955776860,U07.1,E1392,56.0,F,NaN,NCT04699747
3,9704756583,E44.0,NaN,47.0,F,NaN,NCT04699747
4,3353246201,G35,96415,36.0,F,NaN,NCT04699747
...,...,...,...,...,...,...,...
60246361,120789064,M47.816,1125F,70.0,F,NaN,NCT04699747
60246362,141117537,J96.90,E0431,70.0,F,NaN,NCT04699747
60246363,132681533,M54.12,97140,59.0,M,NaN,NCT04699747
60246364,30220098,N39.0,87186,NaN,M,NaN,NCT04699747


In [25]:
ms3_nct=['NCT04466150']*60246366

In [26]:
ms3_data.nct_id=ms3_nct

In [27]:
ms3_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1731468274,R20.9,80053,33.0,F,NaN,NCT04466150
1,712193600,Z20.822,99284,65.0,F,NaN,NCT04466150
2,955776860,U07.1,E1392,56.0,F,NaN,NCT04466150
3,9704756583,E44.0,NaN,47.0,F,NaN,NCT04466150
4,3353246201,G35,96415,36.0,F,NaN,NCT04466150
...,...,...,...,...,...,...,...
60246361,120789064,M47.816,1125F,70.0,F,NaN,NCT04466150
60246362,141117537,J96.90,E0431,70.0,F,NaN,NCT04466150
60246363,132681533,M54.12,97140,59.0,M,NaN,NCT04466150
60246364,30220098,N39.0,87186,NaN,M,NaN,NCT04466150


In [28]:
ms3_data.dtypes

claim_id                   object
diag_cd                    object
origl_prod_svc_qlfr_cd     object
pat_age_yr_nbr            float64
pat_gender_cd              object
product_id                 object
nct_id                     object
dtype: object

In [29]:
mg_eligible = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/mg_eligible.csv', dtype={'patient_id':str})

In [30]:
mg_eligible

,patient_id
0,1015092620
1,92934779
2,1758573595
3,1259818121
4,418349371
...,...
1935,1443518791
1936,944710513
1937,62029906
1938,141814680


In [31]:
sma_eligible = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/sma_eligible.csv', dtype={'patient_id':str})

In [32]:
sma_eligible

,patient_id
0,1031465873
1,1161301033
2,1442099305
3,240460054
4,3325511182
...,...
128,632609211
129,8509914383
130,8737871071
131,896223850


In [33]:
ms1_eligible = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms1_eligible.csv', dtype={'patient_id':str})

In [34]:
ms1_eligible

,patient_id
0,1000080414
1,1016168770
2,1017226051
3,10736891393
4,1190678921
...,...
1706,513222041
1707,550198806
1708,683276850
1709,711960061


In [35]:
ms2_eligible = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms2_eligible.csv', dtype={'patient_id':str})

In [36]:
ms2_eligible

,patient_id
0,10579973611
1,10903383902
2,10563685088
3,7825845811
4,11028492446
...,...
293,10395315444
294,11259272091
295,517646763
296,11095954557


In [37]:
ms3_eligible = pd.read_csv('s3://cmg-oasis-prod-sandbox-commercial-analyst/sandbox_xportfolio_commercial_analyst/cmg_data_science_nlp/clinical_trials_matching/data/final_2/ehr_data/ms3_eligible.csv', dtype={'patient_id':str})

In [38]:
ms3_eligible

,patient_id
0,10791748816
1,1295306268
2,1357525191
3,1435608061
4,2618535631
...,...
688,1368885931
689,323121383
690,328525405
691,6018773768


In [39]:
#ehr_data=ehr_data.iloc[:,1:]

In [40]:
mg_df = pd.DataFrame(columns=['claim_id','diag_cd','origl_prod_svc_qlfr_cd','pat_age_yr_nbr','pat_gender_cd','product_id','nct_id'])

In [41]:
mg_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id


In [42]:
mg_list=mg_eligible.patient_id.tolist()

In [44]:
for i in mg_list:
    print(mg_list.index(i))
    temp_df = mg_data.loc[mg_data['claim_id'] == i]
    mg_df=pd.concat([mg_df,temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939


In [45]:
len(mg_list)

1940

In [46]:
len(set(mg_list))

1940

In [47]:
mg_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NaN,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NaN,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NaN,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NaN,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NaN,DUMMY_NCT_01
...,...,...,...,...,...,...,...
533253,144522632,M54.10,95908,78.0,M,NaN,DUMMY_NCT_01
533254,144522632,E66.9,99213,78.0,M,NaN,DUMMY_NCT_01
533255,144522632,M79.604,99214,78.0,M,NaN,DUMMY_NCT_01
533256,144522632,M54.16,J1100,78.0,M,63323051610,DUMMY_NCT_01


In [48]:
sma_df = pd.DataFrame(columns=['claim_id','diag_cd','origl_prod_svc_qlfr_cd','pat_age_yr_nbr','pat_gender_cd','product_id','nct_id'])

In [49]:
ms1_df = pd.DataFrame(columns=['claim_id','diag_cd','origl_prod_svc_qlfr_cd','pat_age_yr_nbr','pat_gender_cd','product_id','nct_id'])

In [50]:
ms2_df = pd.DataFrame(columns=['claim_id','diag_cd','origl_prod_svc_qlfr_cd','pat_age_yr_nbr','pat_gender_cd','product_id','nct_id'])

In [51]:
ms3_df = pd.DataFrame(columns=['claim_id','diag_cd','origl_prod_svc_qlfr_cd','pat_age_yr_nbr','pat_gender_cd','product_id','nct_id'])

In [52]:
sma_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id


In [53]:
ms1_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id


In [54]:
ms2_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id


In [55]:
ms3_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id


In [56]:
sma_list=sma_eligible.patient_id.tolist()

In [57]:
ms1_list=ms1_eligible.patient_id.tolist()

In [58]:
ms2_list=ms2_eligible.patient_id.tolist()

In [59]:
ms3_list=ms3_eligible.patient_id.tolist()

In [60]:
len(sma_list)

133

In [61]:
len(ms1_list)

1711

In [62]:
len(ms2_list)

298

In [63]:
len(ms3_list)

693

In [65]:
for i in sma_list:
    print(sma_list.index(i))
    temp_df = sma_data.loc[sma_data['claim_id'] == i]
    sma_df=pd.concat([sma_df,temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132


In [ ]:
for i in ms1_list:
    print(ms1_list.index(i))
    temp_df = ms1_data.loc[ms1_data['claim_id'] == i]
    ms1_df=pd.concat([ms1_df,temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [67]:
for i in ms2_list:
    print(ms2_list.index(i))
    temp_df = ms2_data.loc[ms2_data['claim_id'] == i]
    ms2_df=pd.concat([ms2_df,temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [68]:
for i in ms3_list:
    print(ms3_list.index(i))
    temp_df = ms3_data.loc[ms3_data['claim_id'] == i]
    ms3_df=pd.concat([ms3_df,temp_df], ignore_index=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [69]:
print('done')

done


In [70]:
2

2

In [71]:
mg_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NaN,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NaN,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NaN,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NaN,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NaN,DUMMY_NCT_01
...,...,...,...,...,...,...,...
533253,144522632,M54.10,95908,78.0,M,NaN,DUMMY_NCT_01
533254,144522632,E66.9,99213,78.0,M,NaN,DUMMY_NCT_01
533255,144522632,M79.604,99214,78.0,M,NaN,DUMMY_NCT_01
533256,144522632,M54.16,J1100,78.0,M,63323051610,DUMMY_NCT_01


In [72]:
sma_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1031465873,G12.0,A4927,42.0,M,NaN,NCT05232929
1,1031465873,R32,T4535,42.0,M,NaN,NCT05232929
2,1031465873,G12.0,A4624,42.0,M,NaN,NCT05232929
3,1031465873,R32,T4535,42.0,M,NaN,NCT05232929
4,1031465873,R39.81,T4535,42.0,M,NaN,NCT05232929
...,...,...,...,...,...,...,...
16049,350684556,J96.10,A7035,24.0,F,NaN,NCT05232929
16050,350684556,J18.9,E1390,23.0,F,NaN,NCT05232929
16051,350684556,Z23,91313,23.0,F,80777028299,NCT05232929
16052,350684556,J98.4,E1390,23.0,F,NaN,NCT05232929


In [73]:
ms1_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1000080414,F33.2,90853,47.0,F,NaN,NCT04699747
1,1000080414,N92.6,76830,47.0,F,NaN,NCT04699747
2,1000080414,F41.9,90838,46.0,F,NaN,NCT04699747
3,1000080414,F33.2,90836,46.0,F,NaN,NCT04699747
4,1000080414,R41.844,90853,47.0,F,NaN,NCT04699747
...,...,...,...,...,...,...,...
77888,757664512,R06.02,93005,53.0,F,NaN,NCT04699747
77889,757664512,Z79.899,82550,53.0,F,NaN,NCT04699747
77890,757664512,Z79.899,36415,53.0,F,NaN,NCT04699747
77891,757664512,R11.0,99218,53.0,F,NaN,NCT04699747


In [74]:
ms2_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,10563685088,G35,A9579,37.0,F,NaN,NCT04650321
1,10563685088,G35,70553,37.0,F,NaN,NCT04650321
2,10563685088,G35,99213,38.0,F,NaN,NCT04650321
3,11028492446,G35,J2350,38.0,F,50242015001,NCT04650321
4,11028492446,G35,96375,38.0,F,NaN,NCT04650321
...,...,...,...,...,...,...,...
257,3284915030,G35,J2930,30.0,M,00009004726,NCT04650321
258,3284915030,G35,S9379,30.0,M,NaN,NCT04650321
259,3284915030,G35,J2350,30.0,M,50242015001,NCT04650321
260,11095954557,G35,96365,53.0,M,NaN,NCT04650321


In [75]:
ms3_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1295306268,J06.9,99203,35.0,M,NaN,NCT04466150
1,1295306268,J06.9,87880,35.0,M,NaN,NCT04466150
2,1357525191,G35,L2330,49.0,F,NaN,NCT04466150
3,1357525191,Z12.11,45378,49.0,F,NaN,NCT04466150
4,1357525191,G35,L1940,49.0,F,NaN,NCT04466150
...,...,...,...,...,...,...,...
36546,328525405,R53.82,99214,29.0,F,NaN,NCT04466150
36547,328525405,E03.9,99214,29.0,F,NaN,NCT04466150
36548,328525405,N92.0,99214,29.0,F,NaN,NCT04466150
36549,328525405,E78.2,99214,29.0,F,NaN,NCT04466150


In [76]:
#missing ehe data

In [77]:
mg_df.isna().sum()

claim_id                       0
diag_cd                        0
origl_prod_svc_qlfr_cd      9664
pat_age_yr_nbr                 0
pat_gender_cd                  0
product_id                513996
nct_id                         0
dtype: int64

In [78]:
sma_df.isna().sum()

claim_id                      0
diag_cd                       0
origl_prod_svc_qlfr_cd       24
pat_age_yr_nbr              334
pat_gender_cd                 0
product_id                15982
nct_id                        0
dtype: int64

In [79]:
ms1_df.isna().sum()

claim_id                      0
diag_cd                       0
origl_prod_svc_qlfr_cd     2269
pat_age_yr_nbr                0
pat_gender_cd                 0
product_id                75056
nct_id                        0
dtype: int64

In [80]:
ms2_df.isna().sum()

claim_id                    0
diag_cd                     0
origl_prod_svc_qlfr_cd      1
pat_age_yr_nbr              0
pat_gender_cd               0
product_id                194
nct_id                      0
dtype: int64

In [81]:
ms3_df.isna().sum()

claim_id                      0
diag_cd                       0
origl_prod_svc_qlfr_cd      886
pat_age_yr_nbr             1085
pat_gender_cd                 0
product_id                33700
nct_id                        0
dtype: int64

In [82]:
#filling NANs

In [83]:
values = {"diag_cd": 'NULL', "origl_prod_svc_qlfr_cd": 'NULL', "product_id": 'NULL'}

In [84]:
mg_df=mg_df.fillna(value=values)

In [85]:
sma_df=sma_df.fillna(value=values)

In [86]:
ms1_df=ms1_df.fillna(value=values)

In [87]:
ms2_df=ms2_df.fillna(value=values)

In [88]:
ms3_df=ms3_df.fillna(value=values)

In [89]:
#missing Values

In [90]:
mg_df.isna().sum()

claim_id                  0
diag_cd                   0
origl_prod_svc_qlfr_cd    0
pat_age_yr_nbr            0
pat_gender_cd             0
product_id                0
nct_id                    0
dtype: int64

In [91]:
sma_df.isna().sum()

claim_id                    0
diag_cd                     0
origl_prod_svc_qlfr_cd      0
pat_age_yr_nbr            334
pat_gender_cd               0
product_id                  0
nct_id                      0
dtype: int64

In [92]:
ms1_df.isna().sum()

claim_id                  0
diag_cd                   0
origl_prod_svc_qlfr_cd    0
pat_age_yr_nbr            0
pat_gender_cd             0
product_id                0
nct_id                    0
dtype: int64

In [93]:
ms2_df.isna().sum()

claim_id                  0
diag_cd                   0
origl_prod_svc_qlfr_cd    0
pat_age_yr_nbr            0
pat_gender_cd             0
product_id                0
nct_id                    0
dtype: int64

In [94]:
ms3_df.isna().sum()

claim_id                     0
diag_cd                      0
origl_prod_svc_qlfr_cd       0
pat_age_yr_nbr            1085
pat_gender_cd                0
product_id                   0
nct_id                       0
dtype: int64

In [95]:
#Drop Duplicates and NANs

In [96]:
mg_df=mg_df.dropna().reset_index(drop=True)

In [97]:
sma_df=sma_df.dropna().reset_index(drop=True)

In [98]:
ms1_df=ms1_df.dropna().reset_index(drop=True)

In [99]:
ms2_df=ms2_df.dropna().reset_index(drop=True)

In [100]:
ms3_df=ms3_df.dropna().reset_index(drop=True)

In [101]:
mg_df=mg_df.drop_duplicates().reset_index(drop=True)

In [102]:
sma_df=sma_df.drop_duplicates().reset_index(drop=True)

In [103]:
ms1_df=ms1_df.drop_duplicates().reset_index(drop=True)

In [104]:
ms2_df=ms2_df.drop_duplicates().reset_index(drop=True)

In [105]:
ms3_df=ms3_df.drop_duplicates().reset_index(drop=True)

In [106]:
mg_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NULL,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NULL,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NULL,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NULL,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NULL,DUMMY_NCT_01
...,...,...,...,...,...,...,...
180774,144522632,I25.10,99213,78.0,M,NULL,DUMMY_NCT_01
180775,144522632,M54.10,95908,78.0,M,NULL,DUMMY_NCT_01
180776,144522632,E66.9,99213,78.0,M,NULL,DUMMY_NCT_01
180777,144522632,M79.604,99214,78.0,M,NULL,DUMMY_NCT_01


In [107]:
sma_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1031465873,G12.0,A4927,42.0,M,NULL,NCT05232929
1,1031465873,R32,T4535,42.0,M,NULL,NCT05232929
2,1031465873,G12.0,A4624,42.0,M,NULL,NCT05232929
3,1031465873,R39.81,T4535,42.0,M,NULL,NCT05232929
4,1031465873,Z93.0,A7520,43.0,M,NULL,NCT05232929
...,...,...,...,...,...,...,...
3543,350684556,Z68.31,T1015,23.0,F,NULL,NCT05232929
3544,350684556,Z01.419,0134A,23.0,F,NULL,NCT05232929
3545,350684556,J96.10,A7035,24.0,F,NULL,NCT05232929
3546,350684556,Z23,91313,23.0,F,80777028299,NCT05232929


In [108]:
ms1_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1000080414,F33.2,90853,47.0,F,NULL,NCT04699747
1,1000080414,N92.6,76830,47.0,F,NULL,NCT04699747
2,1000080414,F41.9,90838,46.0,F,NULL,NCT04699747
3,1000080414,F33.2,90836,46.0,F,NULL,NCT04699747
4,1000080414,R41.844,90853,47.0,F,NULL,NCT04699747
...,...,...,...,...,...,...,...
49835,757664512,R11.0,84484,53.0,F,NULL,NCT04699747
49836,757664512,R00.2,80050,53.0,F,NULL,NCT04699747
49837,757664512,R06.02,93005,53.0,F,NULL,NCT04699747
49838,757664512,R11.0,99218,53.0,F,NULL,NCT04699747


In [109]:
ms2_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,10563685088,G35,A9579,37.0,F,NULL,NCT04650321
1,10563685088,G35,70553,37.0,F,NULL,NCT04650321
2,10563685088,G35,99213,38.0,F,NULL,NCT04650321
3,11028492446,G35,J2350,38.0,F,50242015001,NCT04650321
4,11028492446,G35,96375,38.0,F,NULL,NCT04650321
...,...,...,...,...,...,...,...
215,3284915030,G35,J2930,30.0,M,00009004726,NCT04650321
216,3284915030,G35,S9379,30.0,M,NULL,NCT04650321
217,3284915030,G35,J2350,30.0,M,50242015001,NCT04650321
218,11095954557,G35,96365,53.0,M,NULL,NCT04650321


In [110]:
ms3_df

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1295306268,J06.9,99203,35.0,M,NULL,NCT04466150
1,1295306268,J06.9,87880,35.0,M,NULL,NCT04466150
2,1357525191,G35,L2330,49.0,F,NULL,NCT04466150
3,1357525191,Z12.11,45378,49.0,F,NULL,NCT04466150
4,1357525191,G35,L1940,49.0,F,NULL,NCT04466150
...,...,...,...,...,...,...,...
23258,328525405,R53.82,99214,29.0,F,NULL,NCT04466150
23259,328525405,E03.9,99214,29.0,F,NULL,NCT04466150
23260,328525405,N92.0,99214,29.0,F,NULL,NCT04466150
23261,328525405,E78.2,99214,29.0,F,NULL,NCT04466150


In [111]:
#missing medcial codes

In [112]:
diag_codes_master=set(diag_dict.keys())

In [113]:
proc_codes_master=set(proc_dict.keys())

In [114]:
prod_codes_master=set(prod_dict.keys())

In [115]:
#MG

In [116]:
diag_codes_mg=set(mg_df.diag_cd.tolist())

In [117]:
diag_codes_mg=[x for x in diag_codes_mg if str(x)!='nan']

In [118]:
proc_codes_mg=set(mg_df.origl_prod_svc_qlfr_cd.tolist())

In [119]:
proc_codes_mg=[x for x in proc_codes_mg if str(x)!='nan']

In [120]:
prod_codes_mg=set(mg_df.product_id.tolist())

In [121]:
prod_codes_mg=[x for x in prod_codes_mg if str(x)!='nan']

In [122]:
missing_diag_mg=set(diag_codes_mg).difference(diag_codes_master)

In [123]:
missing_proc_mg=set(proc_codes_mg).difference(proc_codes_master)

In [124]:
missing_prod_mg=set(prod_codes_mg).difference(prod_codes_master)

In [125]:
len(diag_codes_mg)

5087

In [126]:
len(missing_diag_mg)

0

In [127]:
#5008/5087 diag codes missing

In [128]:
len(proc_codes_mg)

3339

In [129]:
len(missing_proc_mg)

1

In [130]:
#0/3339 proc codes missing

In [131]:
missing_proc_mg

{'NULL'}

In [132]:
len(prod_codes_mg)

847

In [133]:
len(missing_prod_mg)

1

In [134]:
#106/847 diag codes missing

In [135]:
#SMA

In [136]:
diag_codes_sma=set(sma_df.diag_cd.tolist())
diag_codes_sma=[x for x in diag_codes_sma if str(x)!='nan']
proc_codes_sma=set(sma_df.origl_prod_svc_qlfr_cd.tolist())
proc_codes_sma=[x for x in proc_codes_sma if str(x)!='nan']
prod_codes_sma=set(sma_df.product_id.tolist())
prod_codes_sma=[x for x in prod_codes_sma if str(x)!='nan']
missing_diag_sma=set(diag_codes_sma).difference(diag_codes_master)
missing_proc_sma=set(proc_codes_sma).difference(proc_codes_master)
missing_prod_sma=set(prod_codes_sma).difference(prod_codes_master)

In [137]:
len(diag_codes_sma)

351

In [138]:
len(missing_diag_sma)

0

In [139]:
#334/351 diag codes missing

In [140]:
len(proc_codes_sma)

486

In [141]:
len(missing_proc_sma)

1

In [142]:
#0/486 proc does missing 

In [143]:
len(prod_codes_sma)

17

In [144]:
len(missing_prod_sma)

1

In [145]:
#0/17 missing prod codes

In [146]:
prod_codes_sma

['49281042250',
 '49281042150',
 '49281051005',
 '57664068357',
 '49281072288',
 '00270131630',
 '60574411401',
 '70461042210',
 '00005200010',
 '00006412101',
 '58160082152',
 '80777028299',
 '00006404741',
 'NULL',
 '00006498101',
 '70461032203',
 '00005197102']

In [147]:
#MS1

In [148]:
diag_codes_ms1=set(ms1_df.diag_cd.tolist())
diag_codes_ms1=[x for x in diag_codes_ms1 if str(x)!='nan']
proc_codes_ms1=set(ms1_df.origl_prod_svc_qlfr_cd.tolist())
proc_codes_ms1=[x for x in proc_codes_ms1 if str(x)!='nan']
prod_codes_ms1=set(ms1_df.product_id.tolist())
prod_codes_ms1=[x for x in prod_codes_ms1 if str(x)!='nan']
missing_diag_ms1=set(diag_codes_ms1).difference(diag_codes_master)
missing_proc_ms1=set(proc_codes_ms1).difference(proc_codes_master)
missing_prod_ms1=set(prod_codes_ms1).difference(prod_codes_master)

In [149]:
len(diag_codes_ms1)

3051

In [150]:
len(missing_diag_ms1)

0

In [151]:
#2995/3051 diag codes missing

In [152]:
len(proc_codes_ms1)

2130

In [153]:
len(missing_proc_ms1)

1

In [154]:
#8/2130 proc codes missing 

In [155]:
len(prod_codes_ms1)

376

In [156]:
len(missing_prod_ms1)

1

In [157]:
#41/376 prod codes missing

In [158]:
#ms2

In [159]:
diag_codes_ms2=set(ms2_df.diag_cd.tolist())
diag_codes_ms2=[x for x in diag_codes_ms2 if str(x)!='nan']
proc_codes_ms2=set(ms2_df.origl_prod_svc_qlfr_cd.tolist())
proc_codes_ms2=[x for x in proc_codes_ms2 if str(x)!='nan']
prod_codes_ms2=set(ms2_df.product_id.tolist())
prod_codes_ms2=[x for x in prod_codes_ms2 if str(x)!='nan']
missing_diag_ms2=set(diag_codes_ms2).difference(diag_codes_master)
missing_proc_ms2=set(proc_codes_ms2).difference(proc_codes_master)
missing_prod_ms2=set(prod_codes_ms2).difference(prod_codes_master)

In [160]:
len(diag_codes_ms2)

25

In [161]:
len(missing_diag_ms2)

0

In [162]:
#23/25 diag codes mising 

In [163]:
len(proc_codes_ms2)

66

In [164]:
len(missing_proc_ms2)

1

In [165]:
#0/66 missing proc codes

In [166]:
len(prod_codes_ms2)

15

In [167]:
len(missing_prod_ms2)

1

In [168]:
#0/15 missing prod codes

In [169]:
diag_codes_ms3=set(ms3_df.diag_cd.tolist())
diag_codes_ms3=[x for x in diag_codes_ms3 if str(x)!='nan']
proc_codes_ms3=set(ms3_df.origl_prod_svc_qlfr_cd.tolist())
proc_codes_ms3=[x for x in proc_codes_ms3 if str(x)!='nan']
prod_codes_ms3=set(ms3_df.product_id.tolist())
prod_codes_ms3=[x for x in prod_codes_ms3 if str(x)!='nan']
missing_diag_ms3=set(diag_codes_ms3).difference(diag_codes_master)
missing_proc_ms3=set(proc_codes_ms3).difference(proc_codes_master)
missing_prod_ms3=set(prod_codes_ms3).difference(prod_codes_master)

In [170]:
len(diag_codes_ms3)

1818

In [171]:
len(missing_diag_ms3)

0

In [172]:
#1773/1883 missing diag codes

In [173]:
len(proc_codes_ms3)

1489

In [174]:
len(missing_proc_ms3)

1

In [175]:
#2/1489 missing proc codes

In [176]:
len(prod_codes_ms3)

291

In [177]:
len(missing_prod_ms3)

1

In [178]:
#24/291 prod codes missing

In [431]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'

In [432]:
ehr_data = pd.concat([mg_df,sma_df, ms1_df,ms2_df,ms3_df], ignore_index=True)

In [433]:
ehr_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NULL,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NULL,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NULL,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NULL,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NULL,DUMMY_NCT_01
...,...,...,...,...,...,...,...
257645,328525405,R53.82,99214,29.0,F,NULL,NCT04466150
257646,328525405,E03.9,99214,29.0,F,NULL,NCT04466150
257647,328525405,N92.0,99214,29.0,F,NULL,NCT04466150
257648,328525405,E78.2,99214,29.0,F,NULL,NCT04466150


In [434]:
#for i in sma_list:
#    temp_df = duplicate_data.loc[duplicate_data['claim_id'] == i]
#    df2=pd.concat([df2,temp_df], ignore_index=True)

In [435]:
#df2

In [436]:
#ehr_data=ehr_data.iloc[:10000]

In [437]:
#ehr_data = pd.concat([df1, df2], ignore_index=True)

In [438]:
ehr_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NULL,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NULL,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NULL,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NULL,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NULL,DUMMY_NCT_01
...,...,...,...,...,...,...,...
257645,328525405,R53.82,99214,29.0,F,NULL,NCT04466150
257646,328525405,E03.9,99214,29.0,F,NULL,NCT04466150
257647,328525405,N92.0,99214,29.0,F,NULL,NCT04466150
257648,328525405,E78.2,99214,29.0,F,NULL,NCT04466150


In [439]:
#missing values

In [440]:
ehr_data.isna().sum()

claim_id                  0
diag_cd                   0
origl_prod_svc_qlfr_cd    0
pat_age_yr_nbr            0
pat_gender_cd             0
product_id                0
nct_id                    0
dtype: int64

In [441]:
#d_new

In [442]:
#d_new1=d_new.groupby('claim_id')['pat_age_yr_nbr'].apply(lambda x:statistics.mode(x)).reset_index()

In [443]:
#d_new1

In [444]:
#for i,j in d_new.iterrows():
#    print(i)
#    j['pat_age_yr_nbr']=d_new1[d_new1['claim_id']==j['claim_id']]['pat_age_yr_nbr']

In [445]:
print('fone')

fone


In [446]:
ehr_data=ehr_data.drop_duplicates().reset_index(drop=True)

In [447]:
ehr_data

,claim_id,diag_cd,origl_prod_svc_qlfr_cd,pat_age_yr_nbr,pat_gender_cd,product_id,nct_id
0,1015092620,R49.0,92520,23.0,F,NULL,DUMMY_NCT_01
1,1015092620,R13.10,99215,23.0,F,NULL,DUMMY_NCT_01
2,1015092620,Z00.00,87389,23.0,F,NULL,DUMMY_NCT_01
3,1015092620,G70.01,99233,23.0,F,NULL,DUMMY_NCT_01
4,1015092620,R07.89,93010,23.0,F,NULL,DUMMY_NCT_01
...,...,...,...,...,...,...,...
257645,328525405,R53.82,99214,29.0,F,NULL,NCT04466150
257646,328525405,E03.9,99214,29.0,F,NULL,NCT04466150
257647,328525405,N92.0,99214,29.0,F,NULL,NCT04466150
257648,328525405,E78.2,99214,29.0,F,NULL,NCT04466150


In [448]:
#mg_df

In [449]:
#mg_df=mg_df.drop_duplicates().reset_index(drop=True)

In [450]:
#mg_df

In [451]:
#ehr_data.isna().sum()

In [452]:
#ehr_data=ehr_data.dropna().reset_index(drop=True)

In [453]:
#ehr_data.claim_id.value_counts()

In [454]:
claim_trial_dict = {}
total_trials = []
#claim_trial_dict2 = {}
#total_trials2 = []

In [455]:
for idx, row in ehr_data.iterrows():
    if row['claim_id'] not in claim_trial_dict:
        claim_trial_dict[row['claim_id']] = [row['nct_id']]
    elif row['nct_id'] not in claim_trial_dict[row['claim_id']]:
        claim_trial_dict[row['claim_id']].append(row['nct_id'])
    if row['nct_id'] not in total_trials:
        total_trials.append(row['nct_id'])

In [456]:
#for idx, row in mg_df.iterrows():
#    if row['claim_id'] not in claim_trial_dict2:
#        claim_trial_dict2[row['claim_id']] = [row['nct_id']]
#    elif row['nct_id'] not in claim_trial_dict2[row['claim_id']]:
#        claim_trial_dict2[row['claim_id']].append(row['nct_id'])
#    if row['nct_id'] not in total_trials:
#        total_trials.append(row['nct_id'])

In [457]:
#pickle.dump(ehr_data,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/mg_sma_1/ehr_data/final_dataframe', 'wb'))

In [458]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'

In [459]:
pickle.dump(claim_trial_dict,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/claim_trial_dict', 'wb'))
#pickle.dump(claim_trial_dict2,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/ms_mg_sma_2/ehr_data/claim_trial_dict2', 'wb'))

In [460]:
claim_trial_dict

{'1015092620': ['DUMMY_NCT_01'],
 '92934779': ['DUMMY_NCT_01'],
 '1758573595': ['DUMMY_NCT_01'],
 '1259818121': ['DUMMY_NCT_01'],
 '418349371': ['DUMMY_NCT_01'],
 '779303438': ['DUMMY_NCT_01'],
 '329684292': ['DUMMY_NCT_01'],
 '558354417': ['DUMMY_NCT_01'],
 '380096112': ['DUMMY_NCT_01'],
 '71567613': ['DUMMY_NCT_01'],
 '1446471968': ['DUMMY_NCT_01'],
 '133285751': ['DUMMY_NCT_01'],
 '245053962': ['DUMMY_NCT_01'],
 '213313760': ['DUMMY_NCT_01'],
 '10820804324': ['DUMMY_NCT_01'],
 '9975298079': ['DUMMY_NCT_01'],
 '1519922258': ['DUMMY_NCT_01'],
 '616274201': ['DUMMY_NCT_01'],
 '656890220': ['DUMMY_NCT_01'],
 '1140400803': ['DUMMY_NCT_01'],
 '52781241': ['DUMMY_NCT_01'],
 '1348907773': ['DUMMY_NCT_01'],
 '1367881263': ['DUMMY_NCT_01'],
 '170314853': ['DUMMY_NCT_01'],
 '709370764': ['DUMMY_NCT_01'],
 '130222565': ['DUMMY_NCT_01'],
 '643248340': ['DUMMY_NCT_01'],
 '1126322012': ['DUMMY_NCT_01'],
 '1779581204': ['DUMMY_NCT_01'],
 '3953931127': ['DUMMY_NCT_01'],
 '537707091': ['DUMMY_NCT_01'

In [461]:
total_trials

['DUMMY_NCT_01', 'NCT05232929', 'NCT04699747', 'NCT04650321', 'NCT04466150']

In [462]:
age_list = []
for idx, row in ehr_data.iterrows():
    if row['pat_age_yr_nbr'] > 0:
        age_list.append(row['pat_age_yr_nbr'])
mean_age = np.mean(age_list)
std_age = np.std(age_list)

In [463]:
#age_list2 = []
#for idx, row in mg_df.iterrows():
#    if row['pat_age_yr_nbr'] > 0:
#        age_list2.append(row['pat_age_yr_nbr'])
#mean_age2 = np.mean(age_list2)
#std_age2= np.std(age_list2)

In [464]:
valid_ehr_dataset = []
test_ehr_dataset = []
train_ehr_dataset = []

valid_trial_dataset = []
test_trial_dataset = []
train_trial_dataset = []

valid_demo_dataset = []
test_demo_dataset = []
train_demo_dataset = []

valid_label_dataset = []
test_label_dataset = []
train_label_dataset = []

valid_id_dataset = []
test_id_dataset = []
train_id_dataset = []

train_ec=[]
test_ec=[]
valid_ec=[]

In [465]:
total_claims = list(claim_trial_dict.keys())
idx = list(range(len(total_claims)))
train_idx, test_idx = train_test_split(idx, test_size=0.3, random_state=123, shuffle=True)
train_idx, valid_idx = train_test_split(train_idx, test_size=0.1, random_state=123, shuffle=True)

In [466]:
#total_claims1 = list(claim_trial_dict2.keys())
#test_idx = list(range(len(total_claims1)))
#train_idx, test_idx = train_test_split(idx, test_size=0.3, random_state=123, shuffle=True)
#train_idx, valid_idx = train_test_split(idx, test_size=0.1, random_state=123, shuffle=True)

In [467]:
test_idx

[3342,
 1003,
 2950,
 1751,
 554,
 777,
 1370,
 1918,
 2827,
 2246,
 1940,
 3023,
 2066,
 3217,
 1522,
 148,
 1001,
 326,
 2013,
 84,
 1660,
 3050,
 1064,
 2280,
 571,
 3382,
 1577,
 882,
 646,
 1700,
 746,
 1334,
 3061,
 440,
 2180,
 3181,
 3336,
 1392,
 3362,
 684,
 3194,
 611,
 1984,
 2602,
 3480,
 2501,
 3951,
 966,
 3172,
 1599,
 1609,
 1012,
 934,
 3177,
 72,
 3882,
 2899,
 1791,
 3218,
 2794,
 1083,
 2064,
 3597,
 1364,
 2423,
 1362,
 833,
 4029,
 2965,
 2946,
 43,
 1094,
 1625,
 1422,
 1171,
 537,
 3221,
 890,
 3385,
 2779,
 2060,
 1059,
 706,
 3117,
 3319,
 2942,
 1840,
 3448,
 2604,
 743,
 319,
 967,
 3004,
 385,
 3610,
 3395,
 3556,
 1732,
 4039,
 3772,
 1074,
 1770,
 3698,
 1785,
 1895,
 914,
 1589,
 3866,
 63,
 2427,
 1415,
 1511,
 3596,
 2814,
 1356,
 2661,
 2415,
 2091,
 1715,
 3293,
 591,
 893,
 3682,
 3215,
 2650,
 4014,
 3127,
 3159,
 471,
 4009,
 3442,
 3191,
 2882,
 2375,
 2785,
 709,
 1366,
 1428,
 3414,
 1706,
 198,
 3202,
 2098,
 2720,
 3742,
 189,
 2629,
 1905,


In [468]:
len(total_claims)

4040

In [469]:
len(test_idx)

1212

In [470]:
len(train_idx)

2545

In [471]:
#len(test_idx)

In [472]:
len(valid_idx)

283

In [473]:
diag_dict['A00']

{'lv1': array([-6.93654343e-02,  1.09476157e-01,  9.98483062e-01, -9.99994338e-01,
         9.99894261e-01,  6.10432029e-01, -4.16425616e-02,  7.32376456e-01,
        -5.03778234e-02,  1.09384973e-02,  9.99862373e-01,  9.99993622e-01,
        -9.31318164e-01, -9.81601417e-01, -1.58101603e-01, -8.79722476e-01,
         9.99995351e-01,  2.13671010e-02, -9.99682188e-01,  2.40824625e-01,
         1.02183655e-01, -9.96453524e-01,  4.00934108e-02,  9.99314308e-01,
        -5.34844734e-02,  9.14328620e-02,  9.99988973e-01,  9.99735653e-01,
        -1.72510073e-01, -4.30983603e-02, -1.99200641e-02, -9.99926388e-01,
         9.99040186e-01, -9.99962807e-01,  2.08212975e-02,  1.21888071e-02,
        -9.43158716e-02,  1.19779035e-02,  4.91977543e-01, -9.97053623e-01,
         1.34359553e-01, -7.17920780e-01,  1.87025264e-01,  2.23405752e-03,
         9.99722600e-01,  1.08134942e-02, -1.39061868e-01, -1.20267190e-01,
         2.78932694e-02,  9.94647205e-01,  4.44458015e-02,  9.99846876e-01,
     

In [474]:
diag_dataset = {}
diag_dataset['nan'] = np.zeros((4,768))
for each_diag in diag_dict:
    diag_dataset[each_diag] = np.vstack((diag_dict[each_diag]['lv1'], diag_dict[each_diag]['lv2'], diag_dict[each_diag]['lv3'], diag_dict[each_diag]['lv4']))

In [475]:
prod_dataset = {}
prod_dataset['nan'] = np.zeros((4,768))
for each_prod in prod_dict:
    prod_dataset[each_prod] = np.vstack((prod_dict[each_prod]['lv1'], prod_dict[each_prod]['lv2'], prod_dict[each_prod]['lv3'], prod_dict[each_prod]['lv4']))

proc_dataset = {}
proc_dataset['nan'] = np.zeros((4,768))
for each_proc in proc_dict:
    proc_dataset[each_proc] = np.vstack((proc_dict[each_proc]['lv1'], proc_dict[each_proc]['lv2'], proc_dict[each_proc]['lv3'], proc_dict[each_proc]['lv4']))

In [476]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'

In [477]:
pickle.dump(diag_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/diag_dataset', 'wb'))
pickle.dump(prod_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/prod_dataset', 'wb'))
pickle.dump(proc_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/proc_dataset', 'wb'))

In [478]:
for i in range(len(test_idx)):
    print(i)
    cur_idx = test_idx[i]
    cur_data = ehr_data.loc[ehr_data['claim_id'] == total_claims[cur_idx]]
    initial_flag = True
    for idx, row in cur_data.iterrows():
        if row['diag_cd'] not in diag_dict:
            cur_diag = 'nan'
        else:
            cur_diag = row['diag_cd']
            
        if row['product_id'] not in prod_dict:
            cur_prod = 'nan'
        else:
            cur_prod = row['product_id']
            
        if row['origl_prod_svc_qlfr_cd'] not in proc_dict:
            cur_proc = 'nan'
        else:
            cur_proc = row['origl_prod_svc_qlfr_cd']
            
        if initial_flag == True:
            cur_trial = [row['nct_id']]
            initial_flag = False
            cur_ehr = [[cur_diag,cur_prod,cur_proc]]
            cur_age = 0 if row['pat_age_yr_nbr'] < 0 else (row['pat_age_yr_nbr'] - mean_age) / std_age
            cur_gender = np.array([0,1]) if row['pat_gender_cd'] == 'M' else np.array([1,0])
        else:
            cur_trial.append(row['nct_id'])
            tmp_ehr = [cur_diag,cur_prod,cur_proc]
            cur_ehr.append(tmp_ehr)
    test_ehr_dataset.append(cur_ehr)
    test_demo_dataset.append(np.hstack((np.array(cur_age),cur_gender)))
    cur_trial = list(dict.fromkeys(cur_trial))
    for j in range(len(cur_trial)):
        negative_flag_i = True
        negative_flag_e = True
        for k in range(len(embedding_dict[cur_trial[j]]['inclusion'])):
            test_trial_dataset.append((cur_trial[j], 'i', k))
            test_id_dataset.append(i)
            test_label_dataset.append(0)
            test_ec.append(cur_trial[j]+' '+'IC'+str(k+1))
            
            
            #test_trial_dataset_new()
            
            if negative_flag_i == True:
                
                rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
                for a in range(len(embedding_dict[rand_idx]['inclusion'])):
                    test_id_dataset.append(i)
                    test_trial_dataset.append((rand_idx, 'i', a))
                    test_label_dataset.append(2)
                    test_ec.append(rand_idx+' '+'IC'+str(a+1))
                negative_flag_i = False
                
            #rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
            #rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['inclusion']))])
            #            test_id_dataset.append(i)
            #test_trial_dataset.append((rand_idx, 'i', rand_ec))
            #test_label_dataset.append(2)
            #test_ec.append(rand_idx+' '+'IC'+str(k+1))
        if len(embedding_dict[cur_trial[j]]['exclusion']) > 0:
            for k in range(len(embedding_dict[cur_trial[j]]['exclusion'])):
                test_trial_dataset.append((cur_trial[j], 'e', k))
                test_id_dataset.append(i)
                test_label_dataset.append(1)
                test_ec.append(cur_trial[j]+' '+'EC'+str(k+1))
                
                #rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
                if negative_flag_e == True:
                    if len(embedding_dict[rand_idx]['exclusion']) > 0:
                    
                    #rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['exclusion']))])
                        for b in range(len(embedding_dict[rand_idx]['exclusion'])):
                            test_id_dataset.append(i)
                            test_trial_dataset.append((rand_idx, 'e', b))
                            test_label_dataset.append(2)
                            test_ec.append(rand_idx+' '+'EC'+str(b+1))
                    negative_flag_e = False
                    #test_id_dataset.append(i)
                    #test_trial_dataset.append((rand_idx, 'e', rand_ec))
                    #test_label_dataset.append(2)
                    #test_ec.append(rand_idx+' '+'EC'+str(k+1))
            
        #rand_new = random.choice([h for h in total_trials if h not in cur_trial])
        #rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['inclusion']))])
        #test_id_dataset.append(i)
        #test_trial_dataset.append((rand_idx, 'i', rand_ec))
        #test_label_dataset.append(2)
        #test_ec.append(rand_idx+' '+'IC'+str(k+1))
            
            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [479]:
len(test_ec)

48337

In [480]:
len('    ')

4

In [481]:
len('    ')

4

In [482]:
len(test_trial_dataset)

48337

In [483]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'

In [484]:
pickle.dump(test_ehr_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_ehr_dataset', 'wb'))
pickle.dump(test_demo_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_demo_dataset', 'wb'))
pickle.dump(test_trial_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_trial_dataset', 'wb'))
pickle.dump(test_id_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_id_dataset', 'wb'))
pickle.dump(test_label_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_label_dataset', 'wb'))
pickle.dump(test_ec,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_ec', 'wb'))

In [485]:
test_ehr_dataset

[[['G35', 'nan', '99215']],
 [['H40.033', 'nan', '92083'],
  ['H04.123', 'nan', '92012'],
  ['H40.033', 'nan', '92012'],
  ['G54.0', 'nan', '95912'],
  ['G70.01', 'nan', '95937'],
  ['Z96.651', 'nan', '99213'],
  ['G56.03', 'nan', '99204'],
  ['Z00.00', 'nan', '99387'],
  ['G56.03', 'nan', '95912'],
  ['M17.11', 'nan', '73562'],
  ['M25.561', 'nan', '99213'],
  ['M72.2', 'nan', '99387'],
  ['M85.80', 'nan', '99387'],
  ['H25.13', 'nan', '92012'],
  ['K63.5', 'nan', '99387'],
  ['G70.01', 'nan', '95912'],
  ['M17.11', 'nan', '99213'],
  ['H40.033', 'nan', '92133'],
  ['G56.23', 'nan', '95912']],
 [['G35', 'nan', '80061'],
  ['Z13.31', 'nan', '96127'],
  ['Z15.09', 'nan', '99396'],
  ['F41.0', 'nan', '36415'],
  ['F41.0', 'nan', '80061'],
  ['G35', 'nan', '80050'],
  ['F41.0', 'nan', '93000'],
  ['G35', 'nan', '99396'],
  ['Z00.00', 'nan', '36415'],
  ['Z00.00', 'nan', '99396'],
  ['Z00.00', 'nan', '93000'],
  ['Z15.01', 'nan', '93000'],
  ['G35', 'nan', '36415'],
  ['F41.0', 'nan', '800

In [486]:
for i in range(len(valid_idx)):
    print(i)
    cur_idx = valid_idx[i]
    cur_data = ehr_data.loc[ehr_data['claim_id'] == total_claims[cur_idx]]
    initial_flag = True
    for idx, row in cur_data.iterrows():
        if row['diag_cd'] not in diag_dict:
            cur_diag = 'nan'
        else:
            cur_diag = row['diag_cd']
            
        if row['product_id'] not in prod_dict:
            cur_prod = 'nan'
        else:
            cur_prod = row['product_id']
            
        if row['origl_prod_svc_qlfr_cd'] not in proc_dict:
            cur_proc = 'nan'
        else:
            cur_proc = row['origl_prod_svc_qlfr_cd']
            
        if initial_flag == True:
            cur_trial = [row['nct_id']]
            initial_flag = False
            cur_ehr = [[cur_diag,cur_prod,cur_proc]]
            cur_age = 0 if row['pat_age_yr_nbr'] < 0 else (row['pat_age_yr_nbr'] - mean_age) / std_age
            cur_gender = np.array([0,1]) if row['pat_gender_cd'] == 'M' else np.array([1,0])
        else:
            cur_trial.append(row['nct_id'])
            tmp_ehr = [cur_diag,cur_prod,cur_proc]
            cur_ehr.append(tmp_ehr)
    valid_ehr_dataset.append(cur_ehr)
    valid_demo_dataset.append(np.hstack((np.array(cur_age),cur_gender)))
    cur_trial = list(dict.fromkeys(cur_trial))
    for j in range(len(set(cur_trial))):
        for k in range(len(embedding_dict[cur_trial[j]]['inclusion'])):
            valid_trial_dataset.append((cur_trial[j], 'i', k))
            valid_id_dataset.append(i)
            valid_label_dataset.append(0)
            
            rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
            rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['inclusion']))])
            valid_id_dataset.append(i)
            valid_trial_dataset.append((rand_idx, 'i', rand_ec))
            valid_label_dataset.append(2)
        if len(embedding_dict[cur_trial[j]]['exclusion']) > 0:
            for k in range(len(embedding_dict[cur_trial[j]]['exclusion'])):
                valid_trial_dataset.append((cur_trial[j], 'e', k))
                valid_id_dataset.append(i)
                valid_label_dataset.append(1)
                
                rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
                if len(embedding_dict[rand_idx]['exclusion']) > 0:
                    rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['exclusion']))])
                    valid_id_dataset.append(i)
                    valid_trial_dataset.append((rand_idx, 'e', rand_ec))
                    valid_label_dataset.append(2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [487]:
pickle.dump(valid_ehr_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/valid_ehr_dataset', 'wb'))
pickle.dump(valid_demo_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/valid_demo_dataset', 'wb'))
pickle.dump(valid_trial_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/valid_trial_dataset', 'wb'))
pickle.dump(valid_id_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/valid_id_dataset', 'wb'))
pickle.dump(valid_label_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/valid_label_dataset', 'wb'))

In [488]:
for i in range(len(train_idx)):
    print(i)
    cur_idx = train_idx[i]
    cur_data = ehr_data.loc[ehr_data['claim_id'] == total_claims[cur_idx]]
    initial_flag = True
    for idx, row in cur_data.iterrows():
        if row['diag_cd'] not in diag_dict:
            cur_diag = 'nan'
        else:
            cur_diag = row['diag_cd']
            
        if row['product_id'] not in prod_dict:
            cur_prod = 'nan'
        else:
            cur_prod = row['product_id']
            
        if row['origl_prod_svc_qlfr_cd'] not in proc_dict:
            cur_proc = 'nan'
        else:
            cur_proc = row['origl_prod_svc_qlfr_cd']
            
        if initial_flag == True:
            cur_trial = [row['nct_id']]
            initial_flag = False
            cur_ehr = [[cur_diag,cur_prod,cur_proc]]
            cur_age = 0 if row['pat_age_yr_nbr'] < 0 else (row['pat_age_yr_nbr'] - mean_age) / std_age
            cur_gender = np.array([0,1]) if row['pat_gender_cd'] == 'M' else np.array([1,0])
        else:
            cur_trial.append(row['nct_id'])
            tmp_ehr = [cur_diag,cur_prod,cur_proc]
            cur_ehr.append(tmp_ehr)
    train_ehr_dataset.append(cur_ehr)
    train_demo_dataset.append(np.hstack((np.array(cur_age),cur_gender)))
    cur_trial = list(dict.fromkeys(cur_trial))
    for j in range(len(set(cur_trial))):
        for k in range(len(embedding_dict[cur_trial[j]]['inclusion'])):
            train_trial_dataset.append((cur_trial[j], 'i', k))
            train_id_dataset.append(i)
            train_label_dataset.append(0)
            
            rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
            rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['inclusion']))])
            train_id_dataset.append(i)
            train_trial_dataset.append((rand_idx, 'i', rand_ec))
            train_label_dataset.append(2)
        if len(embedding_dict[cur_trial[j]]['exclusion']) > 0:
            for k in range(len(embedding_dict[cur_trial[j]]['exclusion'])):
                train_trial_dataset.append((cur_trial[j], 'e', k))
                train_id_dataset.append(i)
                train_label_dataset.append(1)
                
                rand_idx = random.choice([h for h in total_trials if h not in cur_trial])
                if len(embedding_dict[rand_idx]['exclusion']) > 0:
                    rand_ec = random.choice([h for h in range(len(embedding_dict[rand_idx]['exclusion']))])
                    train_id_dataset.append(i)
                    train_trial_dataset.append((rand_idx, 'e', rand_ec))
                    train_label_dataset.append(2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


In [489]:
pickle.dump(train_ehr_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/train_ehr_dataset', 'wb'))
pickle.dump(train_demo_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/train_demo_dataset', 'wb'))
pickle.dump(train_trial_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/train_trial_dataset', 'wb'))
pickle.dump(train_id_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/train_id_dataset', 'wb'))
pickle.dump(train_label_dataset,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/train_label_dataset', 'wb'))

In [490]:
total_claims

['1015092620',
 '92934779',
 '1758573595',
 '1259818121',
 '418349371',
 '779303438',
 '329684292',
 '558354417',
 '380096112',
 '71567613',
 '1446471968',
 '133285751',
 '245053962',
 '213313760',
 '10820804324',
 '9975298079',
 '1519922258',
 '616274201',
 '656890220',
 '1140400803',
 '52781241',
 '1348907773',
 '1367881263',
 '170314853',
 '709370764',
 '130222565',
 '643248340',
 '1126322012',
 '1779581204',
 '3953931127',
 '537707091',
 '32373050',
 '6872623843',
 '3385018131',
 '326548162',
 '356182828',
 '324342836',
 '679246395',
 '1245319112',
 '902360997',
 '561553128',
 '1893121026',
 '75638060',
 '279216853',
 '8674526466',
 '352435729',
 '1286423773',
 '410161979',
 '914250867',
 '140711002',
 '1281130801',
 '620238493',
 '300429964',
 '1862522584',
 '931861947',
 '620379404',
 '600697718',
 '1463065709',
 '1785670962',
 '1752880099',
 '388686162',
 '1433869997',
 '679536507',
 '1334202098',
 '411426712',
 '2140189106',
 '2857788246',
 '82695830',
 '324220468',
 '430263285

In [491]:
test_patient_list=list(pd.Series(total_claims).iloc[test_idx])

In [492]:
len(test_patient_list)

1212

In [493]:
test_patient_list

['586479297',
 '2489685398',
 '1253278589',
 '352843996',
 '2580932478',
 '686967745',
 '2185526957',
 '1277264297',
 '431486886',
 '210988421',
 '1031465873',
 '179558900',
 '6720829115',
 '3945677724',
 '5363126320',
 '2164781033',
 '711653736',
 '874933912',
 '561785825',
 '1183677425',
 '1393460151',
 '690637650',
 '37046336',
 '1088096670',
 '76469468',
 '187117834',
 '4485504111',
 '3748288589',
 '312206755',
 '807095412',
 '6502091647',
 '794489101',
 '1033700548',
 '321081681',
 '1737865060',
 '1003446102',
 '533431556',
 '3298386729',
 '9850784289',
 '2097013753',
 '144854750',
 '283946267',
 '9458365602',
 '672694295',
 '632968939',
 '1098527685',
 '185895181',
 '1444836915',
 '6658386629',
 '1667251294',
 '2213983333',
 '1509251653',
 '6174893531',
 '83812798',
 '1634318978',
 '1233969241',
 '1192420151',
 '59922492',
 '4845818169',
 '1245863266',
 '437905814',
 '620371192',
 '3225971368',
 '322309401',
 '2706410192',
 '680494421',
 '651870296',
 '299349261',
 '1675895912',


In [494]:
len(list(set(test_patient_list)))

1212

In [495]:
pickle.dump(test_patient_list,open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/ehr_data/test_patient_list', 'wb'))


In [496]:
testpatlist = pickle.load(open('/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2/ehr_data/test_patient_list','rb'))
testpatlist

['586479297',
 '2489685398',
 '1253278589',
 '352843996',
 '2580932478',
 '686967745',
 '2185526957',
 '1277264297',
 '431486886',
 '210988421',
 '1031465873',
 '179558900',
 '6720829115',
 '3945677724',
 '5363126320',
 '2164781033',
 '711653736',
 '874933912',
 '561785825',
 '1183677425',
 '1393460151',
 '690637650',
 '37046336',
 '1088096670',
 '76469468',
 '187117834',
 '4485504111',
 '3748288589',
 '312206755',
 '807095412',
 '6502091647',
 '794489101',
 '1033700548',
 '321081681',
 '1737865060',
 '1003446102',
 '533431556',
 '3298386729',
 '9850784289',
 '2097013753',
 '144854750',
 '283946267',
 '9458365602',
 '672694295',
 '632968939',
 '1098527685',
 '185895181',
 '1444836915',
 '6658386629',
 '1667251294',
 '2213983333',
 '1509251653',
 '6174893531',
 '83812798',
 '1634318978',
 '1233969241',
 '1192420151',
 '59922492',
 '4845818169',
 '1245863266',
 '437905814',
 '620371192',
 '3225971368',
 '322309401',
 '2706410192',
 '680494421',
 '651870296',
 '299349261',
 '1675895912',


In [497]:
#testpatlist==test_patient_list

In [498]:
train_trial_dataset

[('NCT04699747', 'i', 0),
 ('NCT04650321', 'i', 0),
 ('NCT04699747', 'i', 1),
 ('NCT04650321', 'i', 6),
 ('NCT04699747', 'e', 0),
 ('DUMMY_NCT_01', 'e', 2),
 ('NCT04699747', 'e', 1),
 ('NCT04466150', 'e', 4),
 ('NCT04699747', 'e', 2),
 ('NCT04466150', 'e', 8),
 ('NCT04699747', 'e', 3),
 ('NCT05232929', 'e', 1),
 ('NCT04699747', 'e', 4),
 ('DUMMY_NCT_01', 'e', 1),
 ('NCT04699747', 'e', 5),
 ('NCT04466150', 'e', 1),
 ('NCT04699747', 'e', 6),
 ('DUMMY_NCT_01', 'e', 2),
 ('NCT04699747', 'e', 7),
 ('DUMMY_NCT_01', 'e', 0),
 ('NCT04699747', 'e', 8),
 ('NCT05232929', 'e', 1),
 ('NCT04699747', 'e', 9),
 ('NCT04650321', 'e', 2),
 ('NCT04699747', 'e', 10),
 ('NCT05232929', 'e', 1),
 ('NCT04699747', 'e', 11),
 ('DUMMY_NCT_01', 'e', 1),
 ('NCT04699747', 'e', 12),
 ('NCT05232929', 'e', 0),
 ('NCT04699747', 'e', 13),
 ('NCT04466150', 'e', 4),
 ('NCT04699747', 'e', 14),
 ('NCT05232929', 'e', 0),
 ('NCT04699747', 'e', 15),
 ('DUMMY_NCT_01', 'e', 1),
 ('NCT04699747', 'e', 16),
 ('NCT04650321', 'e', 1),

In [499]:
test_ec

['NCT04699747 IC1',
 'DUMMY_NCT_01 IC1',
 'DUMMY_NCT_01 IC2',
 'DUMMY_NCT_01 IC3',
 'DUMMY_NCT_01 IC4',
 'DUMMY_NCT_01 IC5',
 'DUMMY_NCT_01 IC6',
 'DUMMY_NCT_01 IC7',
 'DUMMY_NCT_01 IC8',
 'DUMMY_NCT_01 IC9',
 'DUMMY_NCT_01 IC10',
 'DUMMY_NCT_01 IC11',
 'DUMMY_NCT_01 IC12',
 'DUMMY_NCT_01 IC13',
 'DUMMY_NCT_01 IC14',
 'DUMMY_NCT_01 IC15',
 'DUMMY_NCT_01 IC16',
 'DUMMY_NCT_01 IC17',
 'DUMMY_NCT_01 IC18',
 'DUMMY_NCT_01 IC19',
 'DUMMY_NCT_01 IC20',
 'DUMMY_NCT_01 IC21',
 'DUMMY_NCT_01 IC22',
 'DUMMY_NCT_01 IC23',
 'DUMMY_NCT_01 IC24',
 'DUMMY_NCT_01 IC25',
 'NCT04699747 IC2',
 'NCT04699747 EC1',
 'DUMMY_NCT_01 EC1',
 'DUMMY_NCT_01 EC2',
 'DUMMY_NCT_01 EC3',
 'NCT04699747 EC2',
 'NCT04699747 EC3',
 'NCT04699747 EC4',
 'NCT04699747 EC5',
 'NCT04699747 EC6',
 'NCT04699747 EC7',
 'NCT04699747 EC8',
 'NCT04699747 EC9',
 'NCT04699747 EC10',
 'NCT04699747 EC11',
 'NCT04699747 EC12',
 'NCT04699747 EC13',
 'NCT04699747 EC14',
 'NCT04699747 EC15',
 'NCT04699747 EC16',
 'NCT04699747 EC17',
 'DUMMY_

In [500]:
print('dome today HI')

dome today HI


In [501]:
print('hii')

hii


In [502]:
pwd

'/home/ec2-user/SageMaker/COMPOSE/sai-data-science-clinical-trial-matching/final_2_test/utils'